In [1]:
import os
import matplotlib.pyplot as plt
import scipy.io as sio
import torch
import numpy as np
import pandas as pd
import logging
import re

In [2]:
WD = '/local/meliao/projects/fourier_neural_operator/experiments/09_predict_residuals/'
os.chdir(WD)
print(os.getcwd())

/local/meliao/projects/fourier_neural_operator/experiments/09_predict_residuals


In [3]:
from train_models import SpectralConv1d, FNO1dComplexTime, TimeDataSetResiduals

In [4]:
TRAIN_DF = '/local/meliao/projects/fourier_neural_operator/experiments/09_predict_residuals/results/00_residual_train.txt'
TEST_DF = '/local/meliao/projects/fourier_neural_operator/experiments/09_predict_residuals/results/00_residual_test.txt'
DATA_FP = '/local/meliao/projects/fourier_neural_operator/data/2021-06-24_NLS_data_04_test.mat'
MODEL_FP = '/local/meliao/projects/fourier_neural_operator/experiments/09_predict_residuals/models/00_residual_ep_500'
BASELINE_FP = '/local/meliao/projects/fourier_neural_operator/experiments/08_FNO_pretraining/models/00_pretrain_ep_1000'
# PLOTS_DIR = '/local/meliao/projects/fourier_neural_operator/experiments/09_predict_residuals/plots/'
PLOTS_DIR = 'plots/'
RESULTS_DIR = 'results/'


In [11]:
train_df = pd.read_table(TRAIN_DF)
test_df = pd.read_table(TEST_DF)
fp_train_test = os.path.join(PLOTS_DIR, 'first_train_test.png')

In [5]:
def make_train_test_plot(a_train, a_test, fp=None, title=""):
    fig, ax = plt.subplots(1, 2, sharey=False)
    fig.patch.set_facecolor('white')
#     fig.set_size_inches(6.4, 5.2)


    # a_train and a_test are the time-dependent FNO data. They're in the first column
    ax[0].set_title("Train")
    ax[0].plot(a_train.epoch, a_train.MSE, '-', color='red', label='train')
    ax[1].plot(a_test.epoch, a_test.test_mse, '--', color='red', label='test')
    ax[0].set_xlabel("Epoch", fontsize=13)
    ax[1].set_xlabel("Epoch", fontsize=13)
    # ax[0].legend()
    ax[0].set_yscale('log')

    ax[0].set_ylabel("MSE", fontsize=13)

    # b_train and b_test are the time-dependent. They're in the seecond column
    ax[1].set_title("Test")
    # ax[1].plot(b_train.epoch, b_train.MSE, '-', color='blue', label='train')
    # ax[1].plot(b_test.epoch, b_test.test_mse, '--', color='blue', label='test')
    ax[1].set_xlabel("Epoch", fontsize=13)
    ax[1].set_yscale('log')
    # ax[1].legend(fontsize=13)

    fig.suptitle(title)

#     plt.tight_layout()
    fig.tight_layout(rect=[0, 0.03, 1, 0.95])


    if fp is not None:
        plt.savefig(fp)
    else:
        plt.show()
    plt.close(fig)

In [6]:
# TRAIN_PATTERN = "/local/meliao/projects/fourier_neural_operator/experiments/09_predict_residuals/results/residuals_lr_exp_{}_train.txt"
# TEST_PATTERN = "/local/meliao/projects/fourier_neural_operator/experiments/09_predict_residuals/results/residuals_lr_exp_{}_test.txt"
# for i in [-5, -4, -3, -2.5, -2, -2.5, -1, -0.5, 0, 1]:
#     df_test_i = pd.read_table(TEST_PATTERN.format(i))
#     df_train_i = pd.read_table(TRAIN_PATTERN.format(i))
#     fp_out_i = os.path.join(PLOTS_DIR, 'train_test_{}.png'.format(i))
#     t = "Learning Rate: 10e{}".format(i)
#     make_train_test_plot(df_train_i, df_test_i, fp=fp_out_i, title=t)

results_lst = []
res_re = re.compile("residuals_lr_exp_(.*)_l2_exp_(.*)")
for f in os.listdir(RESULTS_DIR):
    if not f.endswith("_train.txt"):
        continue
    else:
        fname = f.rstrip("_train.txt")
        print("WORKING ON", fname)
        fp_train = os.path.join(RESULTS_DIR, fname + "_train.txt")
        fp_test = os.path.join(RESULTS_DIR, fname + "_test.txt")
        title = fname.split('/')[-1]
        df_train_i = pd.read_table(fp_train)
        df_test_i = pd.read_table(fp_test)
        
        # Find min test error
        dd_i = {}
        dd_i['min_test_error'] = df_test_i.test_mse.mean()
        search_obj = res_re.search(title)
        dd_i['lr_exp'] = float(search_obj.group(1))
        dd_i['l2_exp'] = float(search_obj.group(2))
        
        # Make plot of train/test errors
        fp_out_train_test_plot = os.path.join(PLOTS_DIR, "train_test_" + title + ".png")
        plt_title = "LR=10e{}, L2 Reg=10e{}".format(dd_i['lr_exp'], dd_i['l2_exp'])
        make_train_test_plot(df_train_i, df_test_i, fp=fp_out_train_test_plot, title=plt_title)
        

        print(dd_i)
        # Save result into a list of dictionaries
        results_lst.append(dd_i)
        

WORKING ON residuals_lr_exp_-1.5_l2_exp_-2
{'min_test_error': 0.9546307127518185, 'lr_exp': -1.5, 'l2_exp': -2.0}
WORKING ON residuals_lr_exp_-1.5_l2_exp_-1
{'min_test_error': 0.1203479484678551, 'lr_exp': -1.5, 'l2_exp': -1.0}
WORKING ON residuals_lr_exp_-0.5_l2_exp_-1
{'min_test_error': 45894.14647161016, 'lr_exp': -0.5, 'l2_exp': -1.0}
WORKING ON residuals_lr_exp_-2_l2_exp_1
{'min_test_error': 0.11890807563634742, 'lr_exp': -2.0, 'l2_exp': 1.0}
WORKING ON residuals_lr_exp_-1.5_l2_exp_-5
{'min_test_error': 0.1690894309840027, 'lr_exp': -1.5, 'l2_exp': -5.0}
WORKING ON residuals_lr_exp_-0.5_l2_exp_0
{'min_test_error': 1676829.2165497874, 'lr_exp': -0.5, 'l2_exp': 0.0}
WORKING ON residuals_lr_exp_-3_l2_exp_-5
{'min_test_error': 0.12267313356680216, 'lr_exp': -3.0, 'l2_exp': -5.0}
WORKING ON residuals_lr_exp_-2.5_l2_exp_-0.5
{'min_test_error': 0.11891345250346379, 'lr_exp': -2.5, 'l2_exp': -0.5}
WORKING ON residuals_lr_exp_-0.5_l2_exp_-3
{'min_test_error': 46275736.61305666, 'lr_exp': -

AttributeError: 'NoneType' object has no attribute 'group'

In [18]:
r_df = pd.DataFrame(results_lst).pivot(index='lr_exp', columns='l2_exp', values='min_test_error')
print(r_df.head())

l2_exp      -5.0      -4.0      -3.0      -2.5      -2.0      -1.5      -1.0  \
lr_exp                                                                         
-5.0    0.119486  0.119376  0.119102  0.118997  0.118961  0.118978  0.119099   
-4.0    0.137302  0.132764  0.118929  0.118910  0.118909  0.118921  0.118923   
-3.0    0.122673  0.127400  0.118916  0.118916  0.118919  0.118921  0.118926   
-2.5    0.161237  0.132557  0.118911  0.118914  0.118913  0.118920  0.118920   
-2.0    0.119129  0.119406  0.132846  0.119317  0.124401  0.118915  0.118910   

l2_exp      -0.5       0.0       1.0  
lr_exp                                
-5.0    0.119264  0.119419  0.119631  
-4.0    0.118926  0.118933  0.118952  
-3.0    0.118932  0.118935  0.118942  
-2.5    0.118913  0.118907  0.118906  
-2.0    0.118920  0.118909  0.118908  


In [30]:
def plot_heatmap(values, vals_y, label_y, vals_x, label_x, title, cbarlabel='', fp=None):
    # l1_axis = np.unique(lambda_1) #Lambda 1 on vertical
    # l2_axis = np.unique(lambda_2) #Lambda 2 on horizontal
    # values_arr = values.reshape((l1_axis.shape[0], l2_axis.shape[0]))
#     values = np.log10(values)
    fig, ax = plt.subplots()
    im = ax.imshow(values, origin='lower')
    ax.set_xticks([i for i in range(vals_x.shape[0])])
    ax.set_xticklabels(vals_x)
    # ax.set_xticklabels(["{:.2f}".format(i) for i in l2_axis])
    ax.set_xlabel(label_x, size=14)
    ax.set_yticks([i for i in range(vals_y.shape[0])])
    ax.set_yticklabels(vals_y)
    # ax.set_yticklabels(["{:.2f}".format(i) for i in l1_axis])
    ax.set_ylabel(label_y, size=14)
    cbar = ax.figure.colorbar(im, ax=ax)
    cbar.set_label(cbarlabel)
    # plt.tight_layout()
    plt.title(title)
    if fp is not None:
        plt.savefig(fp)
    else:
        plt.show()
    plt.close(fig)

In [34]:
r_df = r_df[r_df.index < -1.5]
r_df = r_df[[-2., -1.5, -1.0, -0.5, 0., 1.]]

fp_heatmap=os.path.join(PLOTS_DIR, 'heatmap_TDRP_test_errors.png')
plot_heatmap(r_df.values,
            vals_y=r_df.index.values,
            label_y='$log_{10}$(Learning Rate)',
            vals_x=r_df.columns.values,
            label_x='$log_{10}$(L2 reg.)',
            title='TDRP test error',
            cbarlabel='min test error',
            fp=fp_heatmap)

In [4]:
TEST_DF = 'results/residuals_lr_exp_-3_l2_exp_1_test.txt'

In [5]:
test_df = pd.read_table(TEST_DF)
test_df = test_df.sort_values('test_mse')
print(test_df.head())

    epoch  test_l2_normalized_error  test_mse
16    800                  0.287336  0.118325
17    850                  0.287480  0.118507
18    900                  0.287998  0.118624
12    600                  0.287853  0.118632
2     100                  0.287909  0.118643


In [8]:
MODEL_FP = 'models/residuals_lr_exp_-3_l2_exp_1_ep_800'
DATA_FP = '/home/owen/projects/fourier_neural_operator/data/2021-06-24_NLS_data_04_test.mat'
BASELINE_FP = '/home/owen/projects/fourier_neural_operator/experiments/08_FNO_pretraining/models/00_pretrain_ep_1000'

# DATA_FP = '/local/meliao/projects/fourier_neural_operator/data/2021-06-24_NLS_data_04_test.mat'
# MODEL_FP = '/local/meliao/projects/fourier_neural_operator/experiments/09_predict_residuals/models/00_residual_ep_500'
# BASELINE_FP = '/local/meliao/projects/fourier_neural_operator/experiments/08_FNO_pretraining/models/00_pretrain_ep_1000'


In [18]:
make_train_test_plot(train_df, test_df, fp=fp_train_test)

In [16]:
d = sio.loadmat(DATA_FP)
model = torch.load(MODEL_FP, map_location='cpu')
emulator = torch.load(BASELINE_FP, map_location='cpu')

In [17]:
t_dset = TimeDataSetResiduals(d['output'][:, :7], d['t'][:, :7], d['x'], emulator)
t_dloader = torch.utils.data.DataLoader(t_dset, batch_size=1, shuffle=False)

/home/owen/projects/fourier_neural_operator/experiments/09_predict_residuals/train_models.py:155: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  t_tensor = torch.tensor(t_interval, dtype=torch.float).repeat([self.n_batches, 1,1])
/home/owen/projects/fourier_neural_operator/experiments/09_predict_residuals/train_models.py:173: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_input = torch.view_as_real(torch.tensor(X, dtype=torch.cfloat))


In [18]:
def plot_check(x, y, t, preds, resid, fp=None):

    # X has size (grid_size, 3) with the columns being (Re(u_0), Im(u_0), x)
    fig, ax=plt.subplots(nrows=1, ncols=4)
    fig.set_size_inches(15,10) #15, 20 works well
    fig.patch.set_facecolor('white')

    x_real = x[:, 0].flatten()
    x_imag = x[:, 1].flatten()
    # print("X_REAL:", x_real.shape, "X_IMAG:", x_imag.shape)
    # print("PREDS_REAL:", np.real(preds).shape, "PREDS_IMAG:", np.imag(preds).shape)
    # print("Y_REAL:", np.real(y).shape, "Y_IMAG:", np.imag(y).shape)

    ax[0].set_title("$Re(u)$")
    ax[0].plot(x_real, label='Input')
    ax[0].plot(np.real(y), label='Soln') 
    ax[0].plot(np.real(preds), '--', label='Pred')

    ax[0].legend()  

    ax[1].set_title("Residuals: $Re(u)$")
    ax[1].plot(np.real(y) - np.real(preds), color='red', label='actual')
    ax[1].plot(np.real(resid), color='green', label='predicted')
    ax[1].legend()
    
    ax[2].set_title("$Im(u)$")
    ax[2].plot(x_imag, label='Input')
    ax[2].plot(np.imag(y), label='Soln')
    ax[2].plot(np.imag(preds), '--', label='Pred')
    ax[2].legend()

    ax[3].set_title("Residuals: $Im(u)$")

    ax[3].plot(np.imag(y) - np.imag(preds), color='red', label='actual')
    ax[3].plot(np.imag(resid), color='green', label='predicted')

    ax[3].legend()
    
    plt.tight_layout()
    # plt.title("T = {}".format(t))
    if fp is not None:
        plt.savefig(fp)
    else:
        plt.show()
    plt.clf()

In [20]:
model = torch.load(MODEL_FP, map_location=torch.device('cpu'))

In [19]:
n = 0
for x_i, y_i, t_i, preds_i in t_dloader:
    # x_i, y_i, t_i, preds_i = t_dset[i]
    # print(x_i.shape)
    model_resid = model(x_i, t_i)
    fp_i = os.path.join(PLOTS_DIR, 'test_case_{}.png'.format(n))    
    plot_check(x_i[0], y_i[0], t_i.item(), preds_i[0], model_resid[0].detach().numpy(), fp=fp_i)
    n += 1
    if n >= 20:
        break

/home/owen/projects/fourier_neural_operator/experiments/09_predict_residuals/train_models.py:173: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_input = torch.view_as_real(torch.tensor(X, dtype=torch.cfloat))
/home/owen/projects/fourier_neural_operator/experiments/09_predict_residuals/train_models.py:173: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_input = torch.view_as_real(torch.tensor(X, dtype=torch.cfloat))
/home/owen/projects/fourier_neural_operator/experiments/09_predict_residuals/train_models.py:173: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tens

<Figure size 1080x720 with 0 Axes>

<Figure size 1080x720 with 0 Axes>

<Figure size 1080x720 with 0 Axes>

<Figure size 1080x720 with 0 Axes>

...

<Figure size 1080x720 with 0 Axes>

<Figure size 1080x720 with 0 Axes>

<Figure size 1080x720 with 0 Axes>

<Figure size 1080x720 with 0 Axes>

<Figure size 1080x720 with 0 Axes>

In [9]:
def load_data(fp):
    logging.info("Loading data from {}".format(fp))
    data = sio.loadmat(os.path.expanduser(fp))
    return data['output'], data['t']

def load_model(fp, device):
    # Model datatypes are loaded from train_models.py
    model = torch.load(fp, map_location=device)
    return model

def l2_normalized_error(pred, actual):
    """Short summary.

    Parameters
    ----------
    pred : type
        Description of parameter `pred`.
    actual : type
        Description of parameter `actual`.

    Returns
    -------
    types
        Description of returned object.

    """
    errors = pred - actual
    error_norms = torch.linalg.norm(torch.tensor(errors), dim=-1, ord=2)
    actual_norms = torch.linalg.norm(torch.tensor(actual), dim=-1, ord=2)
    normalized_errors = torch.divide(error_norms, actual_norms)
    return normalized_errors.detach().numpy()
def prepare_input(X):
    # X has shape (nbatch, 1, grid_size)
    s = X.shape[-1]
    n_batches = X.shape[0]

    # Convert to tensor
    X_input = torch.view_as_real(torch.tensor(X, dtype=torch.cfloat))

    # FNO code appends the spatial grid to the input as below:
    x_grid = torch.linspace(-np.pi, np.pi, 1024).view(-1,1)
    X_input = torch.cat((X_input, x_grid.repeat(n_batches, 1, 1)), axis=2)

    return X_input

In [10]:
def test_TDRP_predictions(X, t_grid, model, emulator):

    # X has shape (nbatch, n_tsteps, grid_size)
    TEST_KEYS = ['TDRP', 'FNO']
    preds_dd = {i: np.zeros((X.shape[0], X.shape[1]-1, X.shape[2]), dtype=np.cdouble) for i in TEST_KEYS}
    errors_dd = {i: np.zeros((X.shape[0], X.shape[1]-1), dtype=np.double) for i in TEST_KEYS}

    one_tensor = torch.tensor(1, dtype=torch.float).repeat([X.shape[0],1,1])
    half_tensor = torch.tensor(0.5, dtype=torch.float).repeat([X.shape[0],1,1])
    IC_input = prepare_input(X[:,0,:])

    # First input is given by the initial condition
    comp_input_i = prepare_input(X[:,0,:])
    # Iterate along timesteps
    for i in range(t_grid.shape[1]-1):
        SOLN_I = torch.tensor(X[:,i+1,:])
        # First test: composing the model
        comp_preds_i = emulator(comp_input_i, one_tensor).detach().numpy()
        # comp_preds_i = emulator(comp_input_i).detach().numpy()
        preds_dd['FNO'][:,i,:] = comp_preds_i
        comp_input_i = prepare_input(comp_preds_i)
        errors_i = l2_normalized_error(torch.tensor(comp_preds_i), SOLN_I)
        errors_dd['FNO'][:,i] = errors_i

        # Second test: prediction from the TDRP
        # if time_dep_model:
        i_tensor = torch.tensor(t_grid[0,i+1], dtype=torch.float).repeat([X.shape[0],1,1])
        preds_k = model(IC_input, i_tensor).detach().numpy() + comp_preds_i
        preds_dd['TDRP'][:,i,:] = preds_k
        errors_k = l2_normalized_error(torch.tensor(preds_k), SOLN_I)
        errors_dd['TDRP'][:,i] = errors_k
    # for k,v in errors_dd.items():
    #     print("{}: {}: {}".format(k, v.shape, v))
    return preds_dd, errors_dd



In [12]:
X, t_grid = load_data(DATA_FP)
model = load_model(MODEL_FP, torch.device('cpu'))
emulator = load_model(BASELINE_FP, torch.device('cpu'))

preds_dd, errors_dd = test_TDRP_predictions(X, t_grid, model, emulator)

/home/owen/miniconda3/envs/basis_emulators/lib/python3.7/site-packages/ipykernel_launcher.py:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/home/owen/miniconda3/envs/basis_emulators/lib/python3.7/site-packages/ipykernel_launcher.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


In [13]:
def plot_time_errors(errors_dd, t_grid, title, fp):

    n_t_steps = t_grid.shape[1]
    x_vals = t_grid.flatten()
    plt.figure().patch.set_facecolor('white')
    for k, v in errors_dd.items():
        print("{}: {}: {}".format(k, v.shape, v))
        v_means = np.mean(v, axis=0)
        v_stds = np.std(v, axis=0)
        plt.plot(x_vals, v_means, label=k, alpha=0.7)
        plt.fill_between(x_vals,
                            v_means + v_stds,
                            v_means - v_stds,
                            alpha=0.3)
    plt.legend()
    plt.xlabel("Time step")
    plt.xticks(ticks=np.arange(0, n_t_steps),
               labels=make_special_ticks(n_t_steps),
              rotation=45,
              ha='right',
              )
    plt.ylabel("$L_2$-Normalized Errors")
    # plt.yscale('log')
    plt.title(title)
    plt.tight_layout()
    plt.savefig(fp)
    plt.clf()


def make_special_ticks(n):
    s = "$t={} \\ \\to  \\ t={}$"
    return [s.format(i, i+1) for i in range(n)]


In [15]:
fp_time_errors = os.path.join(PLOTS_DIR, 'time_errors.png')
errors_dd_i = {k:np.delete(v, [59], axis=0) for k,v in errors_dd.items()}
plot_time_errors(errors_dd_i, t_grid[:,:-1], "FNO vs. TDRP: Test Dataset", fp_time_errors)

TDRP: (99, 20): [[0.01244915 0.01585605 0.02216385 ... 0.43629836 0.49260077 0.88522732]
 [0.01512696 0.01786236 0.02018981 ... 0.18816753 0.23929786 0.25214812]
 [0.02022892 0.02951047 0.04045442 ... 0.24535782 0.31154048 0.3391816 ]
 ...
 [0.01735274 0.0259438  0.04563707 ... 0.81697395 0.97634489 1.11684803]
 [0.01499148 0.02382209 0.03644001 ... 0.52357611 0.59917947 0.59614398]
 [0.0191487  0.0255491  0.03518951 ... 0.90323849 1.00154942 1.06147841]]
FNO: (99, 20): [[0.01243497 0.01598123 0.02207848 ... 0.43628975 0.49258585 0.88523006]
 [0.01481244 0.01785373 0.02005794 ... 0.18829525 0.23918457 0.25206275]
 [0.02054129 0.03006444 0.04092341 ... 0.24530984 0.31185064 0.33924729]
 ...
 [0.017487   0.0261243  0.04565185 ... 0.81696262 0.97635519 1.11683518]
 [0.01515897 0.02387284 0.03646972 ... 0.52365211 0.59921707 0.59611353]
 [0.01916176 0.02558736 0.03506258 ... 0.90327206 1.00156956 1.06147459]]


<Figure size 432x288 with 0 Axes>

In [2]:
RESULTS_FP = 'second_experiment_results.txt'
results_df = pd.read_table(RESULTS_FP)

In [3]:
print(results_df.head())

   learning_rate  modes  ntest  ntrain      test_mse  train_mse  width
0        0.00001     16   2000   20000  1.627852e+10   3.581666     64
1        0.00001     16   2000   20000  1.627852e+10   3.581113     64
2        0.10000     16   2000   20000  1.627852e+10   3.588306     64
3        0.00001     16   2000   20000  1.623037e+10   3.581877     64
4        0.00001     16   2000   20000  1.623037e+10   3.581964     64


In [4]:
results_df = results_df.sort_values('test_mse')

In [5]:
print(results_df)

    learning_rate  modes  ntest  ntrain      test_mse     train_mse  width
32       0.316228     16   2000   20000  1.623037e+10  1.906141e+01     64
28       0.000100     16   2000   20000  1.623037e+10  3.259620e+00     64
9        0.000010     16   2000   20000  1.623037e+10  3.581531e+00     64
66       0.000010     16   2000   20000  1.623037e+10  3.581822e+00     64
3        0.000010     16   2000   20000  1.623037e+10  3.581877e+00     64
..            ...    ...    ...     ...           ...           ...    ...
42       0.316228     16   2000   20000  1.629357e+10  5.259968e+07     64
33       0.316228     16   2000   20000           NaN           NaN     64
37       0.316228     16   2000   20000           NaN           NaN     64
38       0.316228     16   2000   20000           NaN           NaN     64
41       0.316228     16   2000   20000           NaN           NaN     64

[67 rows x 7 columns]


In [6]:
np.log10(0.316228)

-0.49999967865706474